# Checking and organizing original lab data

In [ ]:
import os
import sys
from functools import reduce
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets

sys.path.insert(0, '../../')
from brickstudy import foundation

In [ ]:
lab_data2 = pd.read_excel('../../secret_data/Brick_lab_20230914.xlsx')

In [ ]:
lab_data2 

In [ ]:
lab_data2['studieID'].unique()

In other lab data (updated lab data)

'BRICK-052', 'BRICK-053', 'BRICK-058', 'BRICK-056', 'BRICK-055',
       'BRICK-054', 'BRICK-027', 'BRICK-031', 'BRICK-028', 'BRICK-024',
       'BRICK-037', 'BRICK-025', 'BRICK-029', 'BRICK-019', 'BRICK-011',
       'BRICK-050', 'BRICK-026', 'BRICK-003', 'BRICK-030', 'BRICK-040',
       'BRICK-002', 'BRICK-014', 'BRICK-004', 'BRICK-042', 'BRICK-047',
       'BRICK-012', 'BRICK-051', 'BRICK-041', 'BRICK-010', 'BRICK-044',
       'BRICK-017', 'BRICK-001', 'BRICK-006', 'BRICK-018', 'BRICK-035',
       'BRICK-049', 'BRICK-016', 'BRICK-015', 'BRICK-048', 'BRICK-023',
       'BRICK-043', 'BRICK-005', 'BRICK-033', 'BRICK-008', 'BRICK-013',
       'BRICK-038', 'BRICK-039

In [ ]:
lab_data2.head(2)

In [ ]:
lab_data2['code_code_original'].unique() 

In [ ]:
lab_data2.columns

In [ ]:
lab_data2.head(3)

In [ ]:
lab_data2['studieID'] = lab_data2['studieID'].astype(str)
lab_data2['studieID'] = lab_data2['studieID'].str.replace("-", "_")

In [ ]:
lab_data2.head(3)

## We want in theory all the patients on thier own line, but not many lines per patient
We will try to accomplish this by splitting the dataset into each code-code-original i.e. each type of lab taken on the assumption we will almost always have one ;ab

In [ ]:
lab_data2.code_code_original.unique()

Notice there is a nan code. This will not work out well for us. It turns out it is in this dataset always the case with Natrium. We will make the code, a string

In [ ]:
codes = lab_data2.code_code_original.unique().astype(str)
codes

In [ ]:
len(codes)

Scientists not interested in ICT, LIP, HEM, ORD8, BAF9 or EX_LUMC therefore we will drop these values

In [ ]:
lab_data2 = lab_data2.drop(lab_data2[lab_data2['code_code_original']== 'ICT'].index)
lab_data2 = lab_data2.drop(lab_data2[lab_data2['code_code_original']== 'LIP'].index)
lab_data2 = lab_data2.drop(lab_data2[lab_data2['code_code_original']== 'HEM'].index)
lab_data2 = lab_data2.drop(lab_data2[lab_data2['code_code_original']== 'ORD8'].index)
lab_data2 = lab_data2.drop(lab_data2[lab_data2['code_code_original']== 'BAF9'].index)
lab_data2 = lab_data2.drop(lab_data2[lab_data2['code_code_original']== 'EX_LUMC'].index)

In [ ]:
lab_data2.tail(3)

Scientist wants to get rid of doubles, i.e. brick 25,12, 08 and 21

Scientist claims all of these are simply doubles. This does not really hold up, let's seperate and let the scientist decide. We have several time points...

In [ ]:
#lab_data2[lab_data2['studieID'] == 'BRICK_008']
lab_data2[lab_data2['studieID'] == 'BRICK_008']['effectiveDateTime'].unique()

In [ ]:
#lab_data2[lab_data2['studieID'] == 'BRICK_012']
lab_data2[lab_data2['studieID'] == 'BRICK_025']['effectiveDateTime'].unique()

In [ ]:
#lab_data2[lab_data2['studieID'] == 'BRICK_021']
lab_data2[lab_data2['studieID'] == 'BRICK_021']['effectiveDateTime'].unique()

In [ ]:
lab_data2[lab_data2['studieID'] == 'BRICK_025']['effectiveDateTime'].unique()

For speed we will assume the scientst wanted the labs in the lab data update, and drop these  out of the data 

In [ ]:
lab_data2_no_ = lab_data2.drop(lab_data2[lab_data2['studieID'] == 'BRICK_008'].index)
lab_data2_no_ = lab_data2_no_.drop(lab_data2[lab_data2['studieID'] == 'BRICK_012'].index)
lab_data2_no_ = lab_data2_no_.drop(lab_data2[lab_data2['studieID'] == 'BRICK_021'].index)
lab_data2_no_ = lab_data2_no_.drop(lab_data2[lab_data2['studieID'] == 'BRICK_025'].index)

In [ ]:
lab_data2_no_.shape

In [ ]:
## Notice we are missing a column compared to the lab data updates? Let's check what it is.

In [ ]:
lab_data2_columns_set = set(lab_data2_no_.columns)

In [ ]:
update_lab_data_columns_set = set(['studieID', 'labdatum_studieID', 'effectiveDateTime', 'issued',
       'code_code_original', 'code_display_original', 'hix_code',
       'valueQuantity_comparator', 'valueQuantity_value', 'valueQuantity_unit',
       'valueString', 'comment', 'interpretation_display',
       'referenceRange_text', 'specimen_type', 'specimen_type_text',
       'container_type', 'sample_note'])
      

In [ ]:
intersection = lab_data2_columns_set.intersection(update_lab_data_columns_set)
intersection

Well, those are the columns we can make the labs into if they should be one merged table.. Keep this in mind for later.

In [ ]:
lab_data2 = lab_data2_no_.reset_index(drop=True)
lab_data2

We will split off the dataframes, then check to make sure we get something for every code

In [ ]:
list_of_data_frames = []
for code in codes:
    prenames = ["code_code_original", code]
    frame_name = "_".join(prenames)
    #print(frame_name)
    frame_name = lab_data2[lab_data2['code_code_original'] == code]
    frame_name = frame_name.add_suffix(code)
    frame_name = frame_name.rename(columns= {frame_name.columns[0]:'studieID'})
    list_of_data_frames.append(frame_name)
    

In [ ]:
len(list_of_data_frames)

In [ ]:
len(codes)

Let's look at an example:

In [ ]:
list_of_data_frames[1].head(3) 

In [ ]:
list_of_data_frames[2].head(3) 

In [ ]:
list_of_data_frames[2].shape

So far so good, we have a frame with in the example above the DBIL. We then tried to do a big reduction; but it produced repeaters

In [ ]:
final_df = reduce(lambda left,right: pd.merge(left,right, on=['studieID'], how='outer'), list_of_data_frames)

In [ ]:
final_df.shape

In [ ]:
#final_df.studieID

In [ ]:
# for number,frame in enumerate(list_of_data_frames):
#     if frame['studieID'].is_unique:
#         print( "Good")
#     else:
#         print(number, "repeaters")

In [ ]:
#final_df

In [ ]:
final_df.head(3)

In [ ]:
final_df.to_csv('../../secret_data/full_labs_EARLIER.csv')

We NO LONGER needed to examine why this happened so we took a look by hand:

Now we want to merge all the lab data into one happy lab dataset.


In [ ]:
updates = pd.read_csv('../../secret_data/full_labs_UPDATES.csv', index_col=0)

In [ ]:
updates.columns

In [ ]:
updates.studieID.unique()

In [ ]:
final_df.studieID.unique()

# There is lots of overlap. Therefore we can make a dataframe of only the updates, and the originals where there is no update

In [ ]:
set_updates = set(updates.studieID.unique())
set_originals = set(final_df.studieID.unique())
overlap = set_originals.intersection(set_updates)
list_overlapers = list(overlap)
#list_overlapers

In [ ]:
set_originals - overlap

In [ ]:
### So we will take only 7, 9 and 32 from the old labs, and add them to the updated labs

In [ ]:
old_labs1 = final_df[final_df['studieID'] == 'BRICK_007' ] 
old_labs2 = final_df[final_df['studieID'] == 'BRICK_009'] 
old_labs3 = final_df[final_df['studieID'] ==  'BRICK_032'] 
old_labs = pd.concat([old_labs1, old_labs2, old_labs3])

In [ ]:
set_old_lab_columns = set(old_labs.columns)

In [ ]:
set_updates_lab_columns = set(updates.columns)
intersectionH = set_old_lab_columns.intersection(set_updates_lab_columns)
columns_here = list(intersectionH)

In [ ]:
## Now we filter these and the others on columsn that intersect between the two
old_labs = old_labs[columns_here]
updated = updates[columns_here]
final_final = pd.concat([old_labs, updated])
final_final['studieID']

In [ ]:
final_final.to_csv('../../secret_data/full_labs_together.csv')